In [1]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
#PATH = "C:\Users\mcveigh\Documents\PythonPC\chrome.exe"
#driver = webdriver.Chrome(PATH)
from selenium.webdriver.chrome.options import Options
import time
import pandas as pd
import re
import os
import sys
import bs4
from bs4 import BeautifulSoup
import requests
import numpy as np

### List of URLs input to search short list manually entered

In [17]:
urls = ['https://www.microbiologyresearch.org/content/journal/ijsem/10.1099/ijsem.0.006404',
       'https://www.microbiologyresearch.org/content/journal/ijsem/10.1099/ijsem.0.006406?emailalert=true',
        'https://www.microbiologyresearch.org/content/journal/ijsem/10.1099/ijsem.0.006401?emailalert=true',
        'https://www.microbiologyresearch.org/content/journal/ijsem/10.1099/ijsem.0.006417?emailalert=true'
       ]

### Input URLS from saved email in html - save as from outlook in htm format

In [3]:
input = (r'IJSEMemail2.htm')

In [18]:
with open (input, encoding = 'unicode_escape') as f:
    content = f.read()
    soup = BeautifulSoup(content, 'html.parser')  
print(soup.prettify())

TypeError: unhashable type: 'list'

In [12]:
urls = []
numberoflinks = 0
for url in soup.findAll( 'a', attrs={'href': re.compile("^https:")}):
    urls.append(url.get('href'))
    numberoflinks = numberoflinks + 1
#print(numberoflinks)
#print(urls)
#remove_list = ['TandC','doi.org', 'myaccount']
urls = [e for e in urls if "TandC" not in e and "doi.org" not in e and "myaccount" not in e]
print(urls)

['https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006417%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7Cede9d3e056284e72eba408dc8c64635f%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638539610385685803%7CUnknown%7CTWFpbGZsb3d8eyJWIjoiMC4wLjAwMDAiLCJQIjoiV2luMzIiLCJBTiI6Ik1haWwiLCJXVCI6Mn0%3D%7C40000%7C%7C%7C&sdata=3IPzb%2BFa%2FxB0gMy87939UdWZLvREPySoF4fnzWRBJQk%3D&reserved=0', 'https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006426%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7Cede9d3e056284e72eba408dc8c64635f%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638539610385701322%7CUnknown%7CTWFpbGZsb3d8eyJWIjoiMC4wLjAwMDAiLCJQIjoiV2luMzIiLCJBTiI6Ik1haWwiLCJXVCI6Mn0%3D%7C40000%7C%7C%7C&sdata=3kRQU%2Bl1KWXJyM%2B7l1yPL2Ex%2FzlEC06BZdrpv28Vgt8%3D&reserved=0', 'https://gcc02.sa

### Selenium web scraper - title and author note (footnote 1)

In [15]:
#Function to scrape headlines using Selenium
def scrape_with_selenium(url):
    global title
    global footnote
    title = []
    footnote = None
    options = Options()
    options.headless = False  # Set to True for headless mode
    driver = webdriver.Chrome(options=options)

    #Navigate to the webpage
    driver.get(url)
    time.sleep(3)

    for element in driver.find_elements(By.CLASS_NAME, "item-meta-data__item-title"):
        #print(element.text)
        title = element.text
        #print(title)
    #for element in driver.find_elements(By.CLASS_NAME, "author-footnote-below-abstract"):get.text() #retrieves all footnotes
    for element in driver.find_elements(By.ID, "FN1"): #retrieves just footnote 1
        #print(element.text)
        footnote = element.text
        #print(footnote)
        #print(type(footnote))
        
    for element in driver.find_elements(By.CLASS_NAME, "tl-main-part title"):
        #print(element.text)
        decription_title = element.text
        print('s2 is', decription_title)
        #print(type(footnote))
        
    #Close the browser window
    driver.quit()
    return title, footnote



### Beautifulsoup webscraper - returns abstract

In [8]:
def scrape_with_beautifulsoup(url):
    global abstract_text
    abstract_text = None
    response = requests.get(url)
    #soup = BeautifulSoup(response.content, "html.parser")
    soup = BeautifulSoup(response.text, "lxml")
    abstract = soup.find(class_= "articleabstract")
    if abstract is not None:
    #print(abstract.get_text())
        abstract_text = abstract.get_text()
        return abstract_text
    else:
        return

### Main body 

In [19]:
pub_df = pd.DataFrame(columns=['PublishedName', 'Accessions', 'Strains','URL'])
pd.set_option('display.max_columns', None)
for url in urls:
    strains = []
    accessions = []
    scrape_with_selenium(url)
    scrape_with_beautifulsoup(url)

    
    #print('title is', title)
    #print('footnote is', footnote)

#find the organism names    
    match = [r'(\S+\s+){2}(?=sp. nov.)', r'(\S+\s+){2}(?=nom. nov.)']
    regex = re.compile(r'\b(' + '|'.join(match) + r')\b')
    orgname = [m.group() for m in regex.finditer(title)]
    print(orgname)

    
#find the accessions - need to add code for scenario where no footnote is found
    #if footnote:
    # got a non-empty string
    #else:
    #    pass
    pattern = [r'[A-Z]{2}\d{6}', r'[A-Z]{4}\d{8}', r'([A-Z]+)(_[A-Z]+)\d{6}', r'[A-Z]{6}\d{9}']
    regex = re.compile(r'\b(' + '|'.join(pattern) + r')\b')
    if footnote is not None:
        accessions = [m.group() for m in regex.finditer(footnote)]
        print(accessions)
    
#find the strains optionally, we could add more search options but may get more junk returned
    strainpattern = [r'(?<=type strain).*'] 
    regex = re.compile(r'\b(' + '|'.join(strainpattern) + r')\b')
    if abstract_text is not None:
        strains = [m.group() for m in regex.finditer(abstract_text)]
        print('strain names', strains)
    
#load data into pandas dataframe
    row_data = [orgname, accessions, strains, url]
    length = len(pub_df)
    pub_df.loc[length] = row_data

['Lacrimispora xylanisolvens ']
strain names []
['Roseateles subflavus ', 'Roseateles aquae ', 'Roseateles violae ']
['JASVDS000000000', 'JASVDS010000000', 'OQ799017', 'JAVXZY000000000', 'JAVXZY010000000', 'OP341190', 'JAUHHC000000000', 'JAUHHC010000000', 'OQ799020']
strain names [' APW6T (=KACC 22877T=TBRC 16606T), Roseateles aquae sp. nov., with the type strain APW11T (=KACC 22878T=TBRC 16607T), and Roseateles violae sp. nov (=KACC 23257T=TBRC 17653T) are respectively proposed']
['Paenibacillus hexagrammi ']
['OM319628', 'CP090978', 'CP090979']
strain names [' of Paenibacillus hexagrammi is YPD9-1T (=KCTC 43424T =LMG 32988T']
['Flavobacterium poyangense ']
['MH392710', 'JAALLN000000000']
strain names [' is JXAS1T (=GDMCC 1.1378T=KCTC 62719T']
['Mangrovimonas cancribranchiae ']
['OR538380', 'OR538379', 'CP136925', 'CP136924']
strain names [' species within the genus Mangrovimonas. The genome sizes were 3.08 and 3.07 Mbp for UG2_1T and UG2_2, respectively, with a G+C content of 33.8\u2

KeyboardInterrupt: 

In [ ]:
pd.set_option('max_colwidth', None)
pub_df['Strains'] = [','.join(map(str, l)) for l in pub_df['Strains']]
pub_d